In [72]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime


# Set options to display maximum rows and columns
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)  # Set display width
pd.set_option('display.max_colwidth', None)  # Set maximum column width


## **Analyzing Our Sales Data**

In [73]:
data_2023 = pd.read_csv("products_2023_complete_1.csv")
data_2023["Date"] = pd.to_datetime(data_2023["Date"])
data_2023.head()

,Date,Product ID,Price ($),Tax ($),Total Price Paid by Client ($),Quantity Ordered,Country
0,2023-01-01,P1001,278.92,27.89,306.81,50,Germany
1,2023-01-01,P1002,430.39,43.04,473.43,89,Canada
2,2023-01-01,P1003,315.55,31.56,347.11,86,Japan
3,2023-01-01,P1004,37.79,3.78,41.57,62,UK
4,2023-01-01,P1005,244.06,24.41,268.47,89,Japan


In [74]:
overall_1 = data_2023[["Price ($)","Tax ($)","Quantity Ordered"]].groupby(data_2023["Date"].dt.month).sum()

# Replace month number with month name
overall_1.index = overall_1.index.map(lambda x: datetime(2023, x, 1).strftime('%B'))


# Create an interactive plot
fig = go.Figure()

# Add traces for Price, Tax, and Quantity Ordered with markers
fig.add_trace(go.Scatter(x=overall_1.index, y=overall_1["Price ($)"], mode='lines+markers', name='Monthly Sales ($)'))
fig.add_trace(go.Scatter(x=overall_1.index, y=overall_1["Tax ($)"], mode='lines+markers', name='Monthly Tax ($)'))
fig.add_trace(go.Scatter(x=overall_1.index, y=overall_1["Quantity Ordered"], mode='lines+markers', name='Quantity Ordered'))

# Update layout
fig.update_layout(
    title='Sales Report of 2023 (Monthly Basis)',
    xaxis_title='Month',
    yaxis_title='Amount ($)',
    legend_title_text='Metrics',
    xaxis=dict(
        tickmode='array',
        tickvals=list(overall_1.index),
        ticktext=list(overall_1.index)
    )
)

fig.show()

In [75]:
# Group by Product ID and Country
product_country_sales = data_2023.groupby([data_2023["Date"].dt.month, 'Country','Product ID'])['Quantity Ordered'].sum().reset_index()

product_country_sales.rename(columns={"Date":"Month"},inplace = True)

product_country_sales.head()

,Month,Country,Product ID,Quantity Ordered
0,1,Australia,P1001,203
1,1,Australia,P1002,123
2,1,Australia,P1003,137
3,1,Australia,P1004,281
4,1,Australia,P1005,175


In [76]:


# Aggregate the data to get the total quantity ordered for each product in each country
product_country_sales2 = data_2023.groupby(['Product ID', 'Country'])['Quantity Ordered'].sum().reset_index()

# Create a stacked bar chart using Plotly Express
fig = px.bar(
    product_country_sales2,
    x='Product ID',
    y='Quantity Ordered',
    color='Country',
    title='Total Quantity Ordered of Each Product in Each Country for 2023',
    labels={'Quantity Ordered': 'Total Quantity Ordered', 'Product ID': 'Product ID'},
    barmode='stack'
)

# Customize the layout
fig.update_layout(
    xaxis_title='Product ID',
    yaxis_title='Total Quantity Ordered',
    legend_title='Country',
    width=1200,
    height=700
)

fig.show()

f:\anaconda\lib\site-packages\plotly\express\_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [78]:

overall_product_country_sales = data_2023.groupby(['Product ID',"Country"])['Total Price Paid by Client ($)'].sum().reset_index()

overall_product_country_sales.head()

,Product ID,Country,Total Price Paid by Client ($)
0,P1001,Australia,13827.00
1,P1001,Canada,12773.17
2,P1001,France,15268.58
3,P1001,Germany,15068.48
4,P1001,Japan,16282.97


In [79]:
import plotly.graph_objects as go

# Create a figure
fig = go.Figure()

# Add traces for each country
countries = overall_product_country_sales['Country'].unique()
for country in countries:
    subset = overall_product_country_sales[overall_product_country_sales['Country'] == country]
    fig.add_trace(go.Bar(
        x=subset['Product ID'],
        y=subset['Total Price Paid by Client ($)'],
        name=country,
        visible=(country == countries[0])
    ))

# Create dropdown menu
dropdown_buttons = [
    dict(
        args=[
            {"visible": [country == c for c in countries]},
            {"annotations": [
                dict(
                    x=0.5,
                    y=1.15,
                    xref='paper',
                    yref='paper',
                    showarrow=False,
                    text="Product Sales Record per Country",
                    font=dict(size=18, family="Times New Roman"),
                ),
                dict(
                    x=0.5,
                    y=1.08,
                    xref='paper',
                    yref='paper',
                    showarrow=False,
                    text=f"Currently Displaying: {country}",
                    font=dict(size=14),
                )
            ]}
        ],
        label=country,
        method="update",
    ) for country in countries
]

# Add a button to show all countries
dropdown_buttons.append(
    dict(
        args=[
            {"visible": [True] * len(countries)},
            {"annotations": [
                dict(
                    x=0.5,
                    y=1.15,
                    xref='paper',
                    yref='paper',
                    showarrow=False,
                    text="Product Sales Record per Country",
                    font=dict(size=18, family="Times New Roman"),
                ),
                dict(
                    x=0.5,
                    y=1.08,
                    xref='paper',
                    yref='paper',
                    showarrow=False,
                    text="Currently Displaying: All Countries",
                    font=dict(size=14),
                )
            ]}
        ],
        label="All",
        method="update",
    )
)

# Update layout with dropdown menu and initial annotations
fig.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=dropdown_buttons,
            direction="down",
            showactive=True,
        )
    ],
    annotations=[
        dict(
            x=0.5,
            y=1.15,
            xref='paper',
            yref='paper',
            showarrow=False,
            text="Product Sales Record per Country",
            font=dict(size=22, family="Times New Roman,bold"),
        ),
        dict(
            x=0.5,
            y=1.08,
            xref='paper',
            yref='paper',
            showarrow=False,
            text="",
            font=dict(size=14),
        )
    ],
    xaxis_title="Product ID",
    yaxis_title="Sales ($)",
    barmode='stack',  # Making the stacked bar chart
    legend_title="Country",  # Add legend title
    width=1400,
    height=600,
)

fig.show()


In [80]:
data_2023.head()


,Date,Product ID,Price ($),Tax ($),Total Price Paid by Client ($),Quantity Ordered,Country
0,2023-01-01,P1001,278.92,27.89,306.81,50,Germany
1,2023-01-01,P1002,430.39,43.04,473.43,89,Canada
2,2023-01-01,P1003,315.55,31.56,347.11,86,Japan
3,2023-01-01,P1004,37.79,3.78,41.57,62,UK
4,2023-01-01,P1005,244.06,24.41,268.47,89,Japan


In [81]:
import pandas as pd

# Assuming data_2023 is your dataset

# Group by "Country" and "Product ID", summing up the "Quantity Ordered"
top_products_by_country = data_2023.groupby(["Country", "Product ID"])["Quantity Ordered"].sum().reset_index()

# Sort by "Country" and the sum of "Quantity Ordered" within each country in descending order
top_products_by_country = top_products_by_country.sort_values(by=["Country", "Quantity Ordered"], ascending=[True, False])

# Get the top 20 products for each country
top_products_by_country = top_products_by_country.groupby("Country").head(5).reset_index(drop=True)

# Display the top products for each country
top_products_by_country.head()


,Country,Product ID,Quantity Ordered
0,Australia,P1014,2842
1,Australia,P1018,2714
2,Australia,P1019,2474
3,Australia,P1013,2340
4,Australia,P1008,2212


In [82]:

# Sort by "Country" and the sum of "Quantity Ordered" within each country in descending order
top_products_by_country = top_products_by_country.sort_values(by=["Country", "Quantity Ordered"], ascending=[True, False])

# Get the top 5 products for each country
top_products_by_country = top_products_by_country.groupby("Country").head(5).reset_index(drop=True)

# Plotting each country's top products separately
for country in top_products_by_country['Country'].unique():
    country_data = top_products_by_country[top_products_by_country['Country'] == country]
    
    fig = px.bar(country_data,
                 x='Product ID',
                 y='Quantity Ordered',
                 color='Product ID',
                 labels={'Quantity Ordered': 'Total Quantity Ordered', 'Product ID': 'Top Product ID'},
                 title=f'Top 5 Ordered Products in {country}',
                 height=400,
                 width=800
                )
    
    # Customize layout
    fig.update_layout(
        xaxis_title='Top Product ID',
        yaxis_title='Total Quantity Ordered',
        legend_title='Product ID',
        barmode='stack',  # Display bars in groups
    )
    
    # Show the figure
    fig.show()

f:\anaconda\lib\site-packages\plotly\express\_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



f:\anaconda\lib\site-packages\plotly\express\_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



f:\anaconda\lib\site-packages\plotly\express\_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



f:\anaconda\lib\site-packages\plotly\express\_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



f:\anaconda\lib\site-packages\plotly\express\_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



f:\anaconda\lib\site-packages\plotly\express\_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



f:\anaconda\lib\site-packages\plotly\express\_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

